In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# **Load in my data from kaggle.**


In [2]:
train = pd.read_csv('../input/climatechangeedsa2020onlinestd/train.csv')
test = pd.read_csv('../input/climatechangeedsa2020onlinestd/test.csv')

In [3]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

# Splitting out the X variable from the target****

In [4]:
y = train['sentiment']
X = train['message']

# Turning text into something a model can read

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

# **Splitting the training data into a training and validation set**

In [6]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

# **Training the model and evaluating using the validation set**

In [7]:
randomfc = RandomForestClassifier()
randomfc.fit(X_train, y_train)
rfc_pred = randomfc.predict(X_val)

# **Checking the performance of our model on the validation set**

In [8]:
f1_score(y_val, rfc_pred, average="macro")

0.5510779426329242

# **Getting our test set ready**

In [9]:
x_test = test['message']
test_vect = vectorizer.transform(x_test)

# **Making predictions on the test set and adding a sentiment column to our original test dataframe**

In [10]:
y_pred = randomfc.predict(test_vect)

In [11]:
test['sentiment'] = y_pred

In [12]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


# **Creating an output csv for submission**

In [13]:
test[['tweetid','sentiment']].to_csv('test_submission.csv', index=False)